In [ ]:
import pandas as pd
import time
import numpy as np

In [ ]:
sales_data = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/f_sales_v_fashion.csv')
sales_data['date'] = pd.to_datetime(sales_data['date'])
sales_data['store'] = sales_data['store'].astype(str)
sales_data = sales_data.rename(columns={'total_sales':'sales'})
sales_data.info()

In [ ]:
print(sales_data['sku'])

In [ ]:
warehouse_data = pd.read_csv('/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/warehouse_stock_fashion.csv')
warehouse_data['valid_to_date'] = warehouse_data['valid_to_date'].replace('2099-12-31', sales_data['date'].max().strftime('%Y-%m-%d'))

In [ ]:
list1 = set(warehouse_data[warehouse_data['stock'] > 0 ]['sku'].unique())

In [ ]:
list2 = set(sales_data[(sales_data['date'] > '2019-01-01') & (sales_data['sales'] > 0)]['sku'].unique())

In [ ]:
list_intersection = list1.intersection(list2)

In [ ]:
skus = list(list_intersection)[:100]

In [ ]:
sales_data = sales_data[sales_data['sku'].isin(skus)]


In [ ]:
warehouse_data.info()

In [ ]:
warehouse_data = warehouse_data[warehouse_data['sku'].isin(skus)]

In [ ]:
warehouse_data['valid_to_date'].max()

In [ ]:
mbew_fashion = pd.read_csv('mbew_fashion.csv')
mbew_fashion['valid_to_date'] = mbew_fashion['valid_to_date'].replace('2099-12-31', sales_data['date'].max().strftime('%Y-%m-%d'))
mbew_fashion['valid_to_date'] = pd.to_datetime(mbew_fashion['valid_to_date'])
mbew_fashion['valid_from_date'] = pd.to_datetime(mbew_fashion['valid_from_date'])
mbew_fashion['item'] = mbew_fashion['sku'].astype(str).str[:12]
mbew_fashion

In [ ]:
mbew_fashion.info()

In [ ]:
mbew_fashion = mbew_fashion[mbew_fashion['sku'].isin(skus)]

In [ ]:
mbew_fashion_no_zero = mbew_fashion[mbew_fashion['stock'] > 0]
mbew_fashion_no_zero['valid_from_date'] = pd.to_datetime(mbew_fashion_no_zero['valid_from_date'])
dict_of_first_month_date_for_sku = {}
i= 0
for sku in mbew_fashion_no_zero['sku'].unique():
    i+=1
    print(str(i) + ' out of ' + str(len(mbew_fashion_no_zero['sku'].unique())))
    data_ = mbew_fashion_no_zero[mbew_fashion_no_zero['sku'] == sku]
    first_date = data_['valid_from_date'].min()
    # put the month in the dict from the first date
    dict_of_first_month_date_for_sku[sku] =  first_date.strftime('%m/%y')

In [ ]:
mbew_fashion['sku'].unique()

In [ ]:
# take MM/YY from first date
first_date.strftime('%m/%y')


In [ ]:
strong_sku = sales_data.groupby(['sku'])['sales'].sum().sort_values(ascending=False).reset_index()

In [ ]:
# import matplotlib.pyplot as plt
# for sku in [100630490000001, 100630490000002, 100630490000003, 100630490000004, 100630490000005, 100630490000006, 100630490000007, 100630490000008, 100630490000009, 100630490000010, 100630490000011, 100630490000012, 100630490000013, 100630490000014, 100630490000015, 100630490000016, 100630490000017, 100630490000018, 100630490000019, 100630490000020, 100630490000021,]:
#     sku_data = sales_data[sales_data['sku'] == sku].groupby('date')['sales'].sum()
#     sku_data = sku_data.reindex(pd.date_range(start=sales_data['date'].min(),
#                                                 # today
#                                                 end=sales_data['date'].max(),
#
#                                                 freq='D')).fillna(0)
#     sku_warehouse = warehouse_data[warehouse_data['sku'].astype(str) == str(sku)].sort_values(by='valid_from_date')
#     sku_warehouse_df_final = pd.DataFrame()
#     for row in sku_warehouse.iterrows():
#         sku_warehouse_df_final = pd.concat([sku_warehouse_df_final, pd.DataFrame({'date':pd.date_range(start=row[1]['valid_from_date'], end=row[1]['valid_to_date'], freq='D'), 'warehouse stock':row[1]['stock']})])
#     sku_warehouse_df_final = sku_warehouse_df_final.set_index('date')
#     relevant_sku_stores_stock = mbew_fashion[mbew_fashion['sku'] == sku]
#     all_stores_sku_stock_data = {}
#     for store in relevant_sku_stores_stock['store'].unique():
#         store_sku_data = relevant_sku_stores_stock[relevant_sku_stores_stock['store'] == store]
#         one_store_stock_sku_all = pd.DataFrame()
#         for row in store_sku_data.iterrows():
#             one_store_stock_sku = pd.DataFrame({'date':pd.date_range(start=row[1]['valid_from_date'],
#                                                                      end=row[1]['valid_to_date'],
#                                                                      freq='D'),
#                                                 'store stock':row[1]['stock']})
#             one_store_stock_sku_all = pd.concat([one_store_stock_sku_all, one_store_stock_sku])
#         one_store_stock_sku_all = one_store_stock_sku_all.set_index('date')
#         all_stores_sku_stock_data[store] = one_store_stock_sku_all
#         # merge all stores with outer join
#     all_stores_final_stock_data = pd.DataFrame()
#     for store in all_stores_sku_stock_data.keys():
#         relevant_store_data_1 = all_stores_sku_stock_data[store]
#         relevant_store_data_1 = relevant_store_data_1.rename(columns={'store stock':'store {}'.format(store) + ' stock'})
#         all_stores_final_stock_data = pd.concat([all_stores_final_stock_data, relevant_store_data_1], axis=1, join='outer')
#     all_store_sum_stock = all_stores_final_stock_data.sum(axis=1).to_frame().rename(columns={0:'all stores stock'})
#     # merge by index
#     sku_data = pd.merge(sku_data, sku_warehouse_df_final, left_index=True, right_index=True, how='left').fillna(0)
#     sku_data = pd.merge(sku_data, all_store_sum_stock, left_index=True, right_index=True, how='left').fillna(0)
#     sku_data_w = sku_data.resample('W').agg({'sales':'sum', 'warehouse stock':'mean', 'all stores stock':'mean'})
#     sku_data_w.plot(figsize=(20,4))
#     plt.title('sku: ' + str(sku))
#
#     plt.show()

In [ ]:
sku_warehouse = warehouse_data[warehouse_data['sku'].astype(str) == str(sku)].sort_values(by='valid_from_date')

In [ ]:
sales_data['sku_store'] = sales_data['sku'].astype(str) + ',' + sales_data['store'].astype(str)
mbew_fashion['sku_store'] = mbew_fashion['sku'].astype(str) + ',' + mbew_fashion['store'].astype(str)

In [ ]:
import numpy as np
import time

start_time = time.time()
mbew_fashion = mbew_fashion.sort_values('valid_from_date')

for store in sales_data['store'].unique():
    print("store:", store)
    len_of_store = sales_data[sales_data['store'] == store]["sku_store"].nunique()

    unique_sku_stores =  sales_data[sales_data['store'] == store]["sku_store"].unique()

    mbew_fashion['valid_from_date'] = pd.to_datetime(mbew_fashion['valid_from_date'])
    mbew_fashion['valid_to_date'] = pd.to_datetime(mbew_fashion['valid_to_date'])
    filtered_mbew_fashion = mbew_fashion[mbew_fashion['sku_store'].isin(unique_sku_stores)]

    # Function to generate date ranges
    def generate_date_ranges(row):
        return pd.date_range(row['valid_from_date'], row['valid_to_date'])

    # Apply function to create date ranges
    df_all_2 = filtered_mbew_fashion.apply(generate_date_ranges, axis=1)

    # Create DataFrame with SKU-store and dates
    df_all_2 = pd.DataFrame({
        'sku_store': np.repeat(filtered_mbew_fashion['sku_store'].values, df_all_2.str.len()),
        'date': np.concatenate(df_all_2.values)  # Convert DatetimeIndex to array for concatenation
    })
    # merge left by ['sku_store', 'date'] and right by ['sku_store', 'valid_to_date']
    df_all_2 = pd.merge(df_all_2, filtered_mbew_fashion[['sku_store','valid_from_date', 'stock']], left_on=['sku_store', 'date'], right_on=['sku_store', 'valid_from_date'], how='left')
    # ffil stock
    df_all_2['stock'] = df_all_2['stock'].ffill()
    df_all_2 = df_all_2.drop(columns=['valid_from_date'])
    df_all_2 = pd.merge(sales_data, df_all_2, on=["sku_store","date"], how="right")
    df_all_2['sku'] = df_all_2['sku_store'].str.split(',').str[0]
    df_all_2['store'] = df_all_2['sku_store'].str.split(',').str[1]
    df_all_2['item'] = df_all_2['sku'].astype(str).str[:12]
    df_all_2['sales'] = df_all_2['sales'].fillna(0)
    df_all_2.to_parquet("/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_4/df_all_store_{}.parquet".format(store))

print("--- %s seconds ---" % (time.time() - start_time))

# generate warehouse data

In [ ]:
sku_unique_in_sales = sales_data['sku'].unique()

In [ ]:
print(warehouse_data[warehouse_data['sku'].isin(skus)])

In [ ]:

start_time = time.time()
warehouse_data['valid_from_date'] = pd.to_datetime(warehouse_data['valid_from_date'])
warehouse_data['valid_to_date'] = pd.to_datetime(warehouse_data['valid_to_date'])
warehouse_data = warehouse_data.sort_values('valid_from_date')
warehouse_data['sku_store'] = warehouse_data['sku'].astype(str) + ',' + warehouse_data['store'].astype(str)
warehouse_data_sample = warehouse_data[warehouse_data['sku'].isin(sku_unique_in_sales)]
for store in ['VZ01']:
    print("store:", store)

    warehouse_data_sample['valid_from_date'] = pd.to_datetime(warehouse_data_sample['valid_from_date'])
    warehouse_data_sample['valid_to_date'] = pd.to_datetime(warehouse_data_sample['valid_to_date'])
    filtered_warehouse_data = warehouse_data_sample

    # Function to generate date ranges
    def generate_date_ranges(row):
        return pd.date_range(row['valid_from_date'], row['valid_to_date'])

    # Apply function to create date ranges
    df_all_2 = filtered_warehouse_data.apply(generate_date_ranges, axis=1)

    # Create DataFrame with SKU-store and dates
    df_all_2 = pd.DataFrame({
        'sku': np.repeat(filtered_warehouse_data['sku'].values, df_all_2.str.len()),
        'date': np.concatenate(df_all_2.values)  # Convert DatetimeIndex to array for concatenation
    })
    df_all_2 = pd.merge(df_all_2, filtered_warehouse_data[['sku','valid_from_date', 'stock']], left_on=['sku', 'date'], right_on=['sku', 'valid_from_date'], how='left')
    # ffil stock
    df_all_2['stock'] = df_all_2['stock'].ffill()
  #  df_all_2 = df_all_2.rename(columns={'stock':'warehouse stock'})
    df_all_2 = df_all_2.drop(columns=['valid_from_date'])

    df_all_2.to_parquet("/Users/guybasson/Desktop/sdatta-nlp/palmers_fashion/reinforcment_codes/datasets_4/df_all_store_{}.parquet".format(store))

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df_all_2

In [ ]:
df_all_2.to_parquet("df_all_store_{}.parquet".format(store))

In [ ]:
df_all_2['sku_store'].unique()

In [ ]:
df_all_2

In [ ]:
warehouse_data_sample[warehouse_data_sample['sku'] == '100657013000006']

In [ ]:
unique_sku_stores

In [ ]:
df_all_2

In [ ]:
df_all_2

In [ ]:
mbew_fashion[mbew_fashion['sku_store'] == '100090812000001,100']

In [ ]:
df_all_2[df_all_2['sku_store'] == '100090812000001,100']

In [ ]:
warehouse_data.info()

In [ ]:
sku_warehouse_df_final

In [ ]:
print(100630490000005)

In [ ]:
sku_data